In [1]:
import requests

def get_wikipedia_views(article: str, year: str):
    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article"
    project = "fr.wikipedia.org"
    access = "all-access"  # (desktop + mobile)
    agent = "user"  # (exclut robots et spiders)
    granularity = "monthly"  # 🔹 Changement ici pour obtenir les vues annuelles

    start_date = f"{year}0101"  # Début de l'année
    end_date = f"{year}1231"  # Fin de l'année
   
    formatted_article = format_article_title(article)
    url = f"{base_url}/{project}/{access}/{agent}/{formatted_article}/{granularity}/{start_date}/{end_date}"
    
    #url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/fr.wikipedia.org/all-access/user/Henri_Hertz/daily/20250201/20250207"
    

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    all_views = 0
    if response.status_code == 200:
        data = response.json()
        for item in data["items"]:
            #all_views += item["timestamp"][:8]: item["views"] for item in data["items"]
            all_views += item["views"]
        
        
    #else:
        #print(f"Erreur {response.status_code}: Impossible de récupérer les données.")

    return all_views


def format_article_title(name: str) -> str:
    return name.replace(" ", "_")

print(get_wikipedia_views("Victor Hugo","2024"))

1027664


In [ ]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd


endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?poet ?name (MIN(?date_birth_raw) AS ?date_birth) (MIN(?date_death_raw) AS ?date_death) 
                (GROUP_CONCAT(DISTINCT ?workLabel; separator=", ") AS ?works) 
                (GROUP_CONCAT(DISTINCT ?movementLabel; separator=", ") AS ?movements)  
WHERE {
  ?poet wdt:P106 wd:Q49757.
  ?poet wdt:P1412 wd:Q150.
  ?poet wdt:P569 ?date_birth_raw.  # Date de naissance obligatoire
  OPTIONAL { ?poet wdt:P570 ?date_death_raw. } # Date de décès optionnelle
  OPTIONAL { 
    ?poet wdt:P800 ?work. 
    ?work rdfs:label ?workLabel.
    FILTER (LANG(?workLabel) = "fr") 
  } # Œuvres principales optionnelles avec leur label en français
  OPTIONAL { 
    ?poet wdt:P135 ?movement. 
    ?movement rdfs:label ?movementLabel.
    FILTER (LANG(?movementLabel) = "fr") 
  } # Mouvements artistiques optionnels avec leur label en français

  ?poet rdfs:label ?name.

  FILTER (LANG(?name) = "fr")
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
GROUP BY ?poet ?name

"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    name = result["name"]["value"]
    nbr_views = get_wikipedia_views(name, "2024")
    
    # Tester si avec api : https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/fr.wikipedia.org/all-access/user/Henri_Hertz/daily/20250201/20250207
    if nbr_views > 10000:
        print(nbr_views)
        array.append(
            (
                result["poet"]["value"],
                name,
                result["date_birth"]["value"],
                result.get("date_death", {}).get("value", None),  # Vérification clé absente
                result.get("works", {}).get("value", ""),  # Vérification clé absente
                result.get("movements", {}).get("value", ""),
                nbr_views,
            )
        )

dataframe = pd.DataFrame(array, columns=["poet", "name", "date_birth", "date_death", "works", "movements", "popularity"])
dataframe = dataframe.astype(
    dtype={
        "poet": "<U200",       # Chaîne Unicode jusqu'à 200 caractères
        "name": "<U200",       # Nom du poète (chaîne)
        "date_birth": "<U200",  # Date de naissance (datetime)
        "date_death": "<U200",  # Date de décès (datetime, peut contenir NaT)
        "works": "<U1000",     # Liste d'œuvres sous forme de chaîne concaténée
        "movements": "<U500",  # Liste de mouvements littéraires sous forme de chaîne concaténée
        "popularity":"int32",
    }
)

dataframe

Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
316546
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
23076
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 404: Impossible de récupérer les données.
Erreur 

,poet,name,date_birth,date_death,works,movements,popularity
0,http://www.wikidata.org/entity/Q214642,Paul Auster,1947-02-03T00:00:00Z,2024-04-30T00:00:00Z,"Léviathan, Seul dans le noir, Trilogie new-yor...",,316546
1,http://www.wikidata.org/entity/Q203952,Antoine de Rivarol,1753-06-26T00:00:00Z,1801-04-11T00:00:00Z,De l'universalité de la langue française,,23076
2,http://www.wikidata.org/entity/Q160534,Jack Kerouac,1922-03-12T00:00:00Z,1969-10-21T00:00:00Z,"Big Sur, Les Souterrains, Les Clochards célest...",Beat Generation,88984
3,http://www.wikidata.org/entity/Q216582,Filippo Tommaso Marinetti,1876-12-22T00:00:00Z,1944-12-02T00:00:00Z,,futurisme,14713
4,http://www.wikidata.org/entity/Q170095,Robert Desnos,1900-07-04T00:00:00Z,1945-06-08T00:00:00Z,,surréalisme,75804
5,http://www.wikidata.org/entity/Q3059438,Etzer Vilaire,1872-04-07T00:00:00Z,1951-05-22T00:00:00Z,,,14255
6,http://www.wikidata.org/entity/Q3105220,Giani Esposito,1930-08-22T00:00:00Z,1974-01-01T00:00:00Z,,,25949
7,http://www.wikidata.org/entity/Q510227,Louise Portal,1950-05-12T00:00:00Z,None,,,11281
8,http://www.wikidata.org/entity/Q200580,Guillaume de Machaut,1300-01-01T00:00:00Z,1377-01-01T00:00:00Z,"Le livre du voir dit, Le jugement du roi de Na...","musique médiévale, musique classique",24358
9,http://www.wikidata.org/entity/Q918784,MC Solaar,1969-03-05T00:00:00Z,None,,,348714


In [7]:
# Convertir data frame en Json : 
import json

def save_dataframe_to_json(dataframe: pd.DataFrame, filename: str, orient: str = "records", indent: int = 4):
    """
    Convertit un DataFrame pandas en JSON et l'enregistre dans un fichier.

    :param dataframe: Le DataFrame à convertir.
    :param filename: Nom du fichier de sortie (ex: "data.json").
    :param orient: Format du JSON (ex: "records", "split", "index", "columns", "values").
    :param indent: Nombre d'espaces pour l'indentation du JSON.
    """
    json_data = dataframe.to_json(orient=orient, indent=indent, force_ascii=False)
    
    with open(filename, "w", encoding="utf-8") as file:
        file.write(json_data)
save_dataframe_to_json(dataframe, "data.json")

In [ ]:
# Affichage camembert sur movements

nombre_couleur = 5
imgfile = Image.open("../../images/flower.jpg")
numarray = numpy.array(imgfile.getdata(), numpy.uint8)
clusters = KMeans(n_clusters=nombre_couleur, n_init=2)
clusters.fit(numarray)
npbins = numpy.arange(0, nombre_couleur + 1)
histogram, _ = numpy.histogram(clusters.labels_, bins=npbins)

sorted_indices = numpy.argsort(histogram)  # Indices triés par fréquence croissante
sorted_histogram = histogram[sorted_indices]
sorted_colors = clusters.cluster_centers_[sorted_indices]

#labels = numpy.unique(clusters.labels_)
labels = numpy.arange(nombre_couleur)
#barlist = plot.bar(labels, histogram[0])

colors_hex = [
    "#%02x%02x%02x"
    % (
        math.ceil(sorted_colors[i][0]),
        math.ceil(sorted_colors[i][1]),
        math.ceil(sorted_colors[i][2]),
    )
    for i in range(nombre_couleur)
]


plot.pie(sorted_histogram, colors=colors_hex)


plot.show()

In [ ]:
# https://fr.wikipedia.org/w/api.php?action=query&format=json&titles=Albert%20Einstein&prop=revisions&rvprop=content&rvsection=0
# lien pour obtenir cadre à droite et première section